In [1]:
import requests
import re
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import unicodedata
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect
import datetime
from datetime import date, timedelta

In [2]:
engine = sqlalchemy.create_engine('mysql+pymysql://root:p@ssw0rd@localhost:3306/mlb_db')

In [165]:

def box_to_sql(year,month,day):
    game_list = box_score(year,month,day)
    batter = requests.get('http://gd2.mlb.com'+ game_list[0] +'/boxscore.json').json()['data']['boxscore']['batting']
    pitcher = requests.get('http://gd2.mlb.com'+ game_list[0] +'/boxscore.json').json()['data']['boxscore']['pitching']
    games_date = 'mlb_'+str(year) +'_'+str(month)+'_'+str(day)
    num_bat = ['a', 'ab', 'ao', 'avg', 'bb', 'bo', 'cs', 'd', 'e', 'fldg', 'go', 'h', 'hbp', 'hr', 'lob', 'obp', 'ops', 'po', 'r', 'rbi', 's_bb', 's_h', 's_hr', 's_r', 's_rbi', 's_so', 'sac', 'sb', 'sf', 'slg', 'so', 't']
    num_p = ['bb', 'bf', 'bs','er', 'game_score', 'h', 'hld', 'hr', 'l', 'np', 'out', 'r', 's', 's_bb', 's_er', 's_h', 's_ip', 's_r', 's_so', 'so', 'sv', 'w']
    lambda_w = lambda row: 1 if row.win == 'true' else 0
    lambda_s = lambda row: 1 if row.save == 'true' else 0
    bats_df = pd.concat([pd.DataFrame(batter[0]['batter']), pd.DataFrame(batter[1]['batter'])], sort = True).reset_index()
    for num in num_bat:
        bats_df[num] = bats_df[num].apply(lambda x: float(x))
    bats_df['points'] = bats_df.hr*6 + bats_df.sb*5 + bats_df.h*4 + bats_df.r*2 + bats_df.rbi*2 - bats_df.ab
    if type(pitcher[0]['pitcher']) == list and type(pitcher[1]['pitcher']) == list:
        pitch_df = pd.concat([pd.DataFrame(pitcher[0]['pitcher']), pd.DataFrame(pitcher[1]['pitcher'])], sort = True).reset_index()
        for num in num_p:
            pitch_df[num] = pitch_df[num].apply(lambda x: float(x))
        pitch_df.win = pitch_df.apply(lambda_w, axis = 1)
        try:
            pitch_df.save = pitch_df.apply(lambda_s, axis = 1)
            pitch_df['points'] = pitch_df.win*6 + pitch_df.save*6 + pitch_df.out + pitch_df.so - (pitch_df.h + pitch_df.bb + pitch_df.er*2)
        except:
            pitch_df['points'] = pitch_df.win*6 + pitch_df.out + pitch_df.so - (pitch_df.h + pitch_df.bb + pitch_df.er*2)
    elif type(pitcher[0]['pitcher']) == dict and type(pitcher[1]['pitcher']) == list:
        pitch_df = pd.concat([pd.DataFrame(pitcher[0]['pitcher'], index = [0]), pd.DataFrame(pitcher[1]['pitcher'])], sort = True).reset_index()
        for num in num_p:
            pitch_df[num] = pitch_df[num].apply(lambda x: float(x))
        pitch_df.win = pitch_df.apply(lambda_w, axis = 1)
        try:
            pitch_df.save = pitch_df.apply(lambda_s, axis = 1)
            pitch_df['points'] = pitch_df.win*6 + pitch_df.save*6 + pitch_df.out + pitch_df.so - (pitch_df.h + pitch_df.bb + pitch_df.er*2)
        except:
            pitch_df['points'] = pitch_df.win*6 + pitch_df.out + pitch_df.so - (pitch_df.h + pitch_df.bb + pitch_df.er*2)
    elif type(pitcher[0]['pitcher']) == list and type(pitcher[1]['pitcher']) == dict:
        pitch_df = pd.concat([pd.DataFrame(pitcher[0]['pitcher']), pd.DataFrame(pitcher[1]['pitcher'], index = [0])], sort = True).reset_index()
        for num in num_p:
            pitch_df[num] = pitch_df[num].apply(lambda x: float(x))
        pitch_df.win = pitch_df.apply(lambda_w, axis = 1)
        try:
            pitch_df.save = pitch_df.apply(lambda_s, axis = 1)
            pitch_df['points'] = pitch_df.win*6 + pitch_df.save*6 + pitch_df.out + pitch_df.so - (pitch_df.h + pitch_df.bb + pitch_df.er*2)
        except:
            pitch_df['points'] = pitch_df.win*6 + pitch_df.out + pitch_df.so - (pitch_df.h + pitch_df.bb + pitch_df.er*2)
    elif type(pitcher[0]['pitcher']) == dict and type(pitcher[1]['pitcher']) == dict:
        pitch_df = pd.concat([pd.DataFrame(pitcher[0]['pitcher'], index = [0]), pd.DataFrame(pitcher[1]['pitcher'], index = [0])], sort = True).reset_index()
        for num in num_p:
            pitch_df[num] = pitch_df[num].apply(lambda x: float(x))
        pitch_df.win = pitch_df.apply(lambda_w, axis = 1)
        try:
            pitch_df.save = pitch_df.apply(lambda_s, axis = 1)
            pitch_df['points'] = pitch_df.win*6 + pitch_df.save*6 + pitch_df.out + pitch_df.so - (pitch_df.h + pitch_df.bb + pitch_df.er*2)
        except:
            pitch_df['points'] = pitch_df.win*6 + pitch_df.out + pitch_df.so - (pitch_df.h + pitch_df.bb + pitch_df.er*2)
    whole_game = bats_df.merge(pitch_df, how = 'outer', on = ['index','bb','h','hr','id','name','name_display_first_last','points','pos','r','s_bb','s_h','s_r','s_so','so'])
    for n in range(1,len(game_list)):
        if game_list[n]:
            batter = requests.get('http://gd2.mlb.com'+ game_list[n] +'/boxscore.json').json()['data']['boxscore']['batting']
            pitcher = requests.get('http://gd2.mlb.com'+ game_list[n] +'/boxscore.json').json()['data']['boxscore']['pitching']
            new_bats_df = pd.concat([pd.DataFrame(batter[0]['batter']), pd.DataFrame(batter[1]['batter'])], sort = True).reset_index()
            for num in num_bat:
                new_bats_df[num] = new_bats_df[num].apply(lambda x: float(x))
            new_bats_df['points'] = new_bats_df.hr*6 + new_bats_df.sb*5 + new_bats_df.h*4 + new_bats_df.r*2 + new_bats_df.rbi*2 - new_bats_df.ab
            if type(pitcher[0]['pitcher']) == list and type(pitcher[1]['pitcher']) == list:
                new_pitch_df = pd.concat([pd.DataFrame(pitcher[0]['pitcher']), pd.DataFrame(pitcher[1]['pitcher'])], sort = True).reset_index()
                for num in num_p:
                    new_pitch_df[num] = new_pitch_df[num].apply(lambda x: float(x))
                new_pitch_df.win = new_pitch_df.apply(lambda_w, axis = 1)
                try:
                    new_pitch_df.save = new_pitch_df.apply(lambda_s, axis = 1)
                    new_pitch_df['points'] = new_pitch_df.win*6 + new_pitch_df.save*6 + new_pitch_df.out + new_pitch_df.so - (new_pitch_df.h + new_pitch_df.bb + new_pitch_df.er*2)
                except:
                    new_pitch_df['points'] = new_pitch_df.win*6 + new_pitch_df.out + new_pitch_df.so - (new_pitch_df.h + new_pitch_df.bb + new_pitch_df.er*2)
            elif type(pitcher[0]['pitcher']) == dict and type(pitcher[1]['pitcher']) == list:
                new_pitch_df = pd.concat([pd.DataFrame(pitcher[0]['pitcher'], index = [0]), pd.DataFrame(pitcher[1]['pitcher'])], sort = True).reset_index()
                for num in num_p:
                    new_pitch_df[num] = new_pitch_df[num].apply(lambda x: float(x))
                new_pitch_df.win = new_pitch_df.apply(lambda_w, axis = 1)
                try:
                    new_pitch_df.save = new_pitch_df.apply(lambda_s, axis = 1)
                    new_pitch_df['points'] = new_pitch_df.win*6 + new_pitch_df.save*6 + new_pitch_df.out + new_pitch_df.so - (new_pitch_df.h + new_pitch_df.bb + new_pitch_df.er*2)
                except:
                    new_pitch_df['points'] = new_pitch_df.win*6 + new_pitch_df.out + new_pitch_df.so - (new_pitch_df.h + new_pitch_df.bb + new_pitch_df.er*2)
            elif type(pitcher[0]['pitcher']) == list and type(pitcher[1]['pitcher']) == dict:
                new_pitch_df = pd.concat([pd.DataFrame(pitcher[0]['pitcher']), pd.DataFrame(pitcher[1]['pitcher'], index = [0])], sort = True).reset_index()
                for num in num_p:
                    new_pitch_df[num] = new_pitch_df[num].apply(lambda x: float(x))
                new_pitch_df.win = new_pitch_df.apply(lambda_w, axis = 1)
                try:
                    new_pitch_df.save = new_pitch_df.apply(lambda_s, axis = 1)
                    new_pitch_df['points'] = new_pitch_df.win*6 + new_pitch_df.save*6 + new_pitch_df.out + new_pitch_df.so - (new_pitch_df.h + new_pitch_df.bb + new_pitch_df.er*2)
                except:
                    new_pitch_df['points'] = new_pitch_df.win*6 + new_pitch_df.out + new_pitch_df.so - (new_pitch_df.h + new_pitch_df.bb + new_pitch_df.er*2)
            elif type(pitcher[0]['pitcher']) == dict and type(pitcher[1]['pitcher']) == dict:
                new_pitch_df = pd.concat([pd.DataFrame(pitcher[0]['pitcher'], index = [0]), pd.DataFrame(pitcher[1]['pitcher'], index = [0])], sort = True).reset_index()
                for num in num_p:
                    new_pitch_df[num] = new_pitch_df[num].apply(lambda x: float(x))
                new_pitch_df.win = new_pitch_df.apply(lambda_w, axis = 1)
                try:
                    new_pitch_df.save = new_pitch_df.apply(lambda_s, axis = 1)
                    new_pitch_df['points'] = new_pitch_df.win*6 + new_pitch_df.save*6 + new_pitch_df.out + new_pitch_df.so - (new_pitch_df.h + new_pitch_df.bb + new_pitch_df.er*2)
                except:
                    new_pitch_df['points'] = new_pitch_df.win*6 + new_pitch_df.out + new_pitch_df.so - (new_pitch_df.h + new_pitch_df.bb + new_pitch_df.er*2)
            whole_new = new_bats_df.merge(new_pitch_df, how = 'outer',  on = ['index','bb','h','hr','id','name','name_display_first_last','points','pos','r','s_bb','s_h','s_r','s_so','so'])
            whole_game = pd.concat([whole_game, whole_new], sort = True)
    whole_game = whole_game.rename(columns = {'out': 'outs'})
    whole_game = whole_game.fillna(value = 0)
    whole_game.to_sql(name = games_date, con = engine, index = False, if_exists = 'replace')

    
def sql_to_pandas(year, month, day):
    boxscore = pd.read_sql_table('mlb_' + str(year) + '_' +str(month) + '_' + str(day), con = engine, schema = 'mlb_db')
    return boxscore

def sql_daily_dict(year, month, day):
    with engine.connect() as con:
        query = 'SELECT ab, bb, er, h, hr, id, name_display_first_last as name, outs, points, pos, r, rbi, save as sv, sb, so, win as w FROM mlb_' +str(year) +'_' + str(month) +'_' +str(day)
        mlb = con.execute(query)

    sdict = {'id': [],
             'name': [],
             'ab': [],
             'h': [],
             'r': [],
             'hr': [],
             'rbi': [],
             'sb': [],
             'w': [],
             'sv': [],
             'so': [],
             'bb': [],
             'outs': [],
             'er': [],
             'pos': [],
             'points': []
            }

    for col in mlb:
        for key in sdict:
            sdict[key].append(col[key])
    return sdict

def linear_search(search_list, target_value):
    for n in range(len(search_list)):
        if search_list[n] == target_value:
            return n

def fantasy_stats(year,month,day,team_dataframe):
    stats_dict = sql_daily_dict(year,month,day)
    team_array = team_list.values
    new_array = []
    for n in range(len(team_array)):
        idx = linear_search(stats_dict['id'], team_array[n][2])
        if idx:
            new_array = np.append(new_array, np.append(team_array[n], stats_dict['points'][idx]))
    return new_array 


In [5]:
box_score(2018,3,29)

['/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_chnmlb_miamlb_1',
 '/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_slnmlb_nynmlb_1',
 '/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_minmlb_balmlb_1',
 '/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_houmlb_texmlb_1',
 '/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_nyamlb_tormlb_1',
 '/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_bosmlb_tbamlb_1',
 '/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_anamlb_oakmlb_1',
 '/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_milmlb_sdnmlb_1',
 '/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_phimlb_atlmlb_1',
 '/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_chamlb_kcamlb_1',
 '/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_sfnmlb_lanmlb_1',
 '/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_clemlb_seamlb_1',
 '/components/ga

In [43]:
batter = requests.get('http://gd2.mlb.com/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_bosmlb_tbamlb_1/boxscore.json').json()['data']['boxscore']['batting']
pitcher = requests.get('http://gd2.mlb.com/components/game/mlb/year_2018/month_03/day_29/gid_2018_03_29_slnmlb_nynmlb_1/boxscore.json').json()['data']['boxscore']['pitching']
pitch_df = pd.concat([pd.DataFrame(pitcher[0]['pitcher']), pd.DataFrame(pitcher[1]['pitcher'])], sort = True).reset_index()
bats_df = pd.concat([pd.DataFrame(batter[0]['batter']), pd.DataFrame(batter[1]['batter'])], sort = True).reset_index()

In [53]:
opening_day = datetime.date(2018,3,29)
last_day = datetime.date(2018,9,30)

In [55]:
team_list = pd.read_csv('team_list.csv')

In [54]:
dd = [opening_day + timedelta(days=x) for x in range((last_day-opening_day).days + 1)]

In [62]:
dd[108]

datetime.date(2018, 7, 15)

In [63]:
for n in range(len(dd)):
    print(n)
    box_to_sql(dd[n].year, dd[n].month, dd[n].day)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182


In [65]:
del team_list['Unnamed: 0']
team_list.id = team_list.id.apply(lambda x: str(x).split('.')[0])

In [74]:
team_list.to_csv('team_list.csv')

In [166]:
stats_6_6_18 = fantasy_stats(2018,6,6,team_list)

In [167]:
stats_6_6_18

array(['Smoak Me A Kingery', 'Gary Sanchez', '596142', 'C', -2.0,
       'Smoak Me A Kingery', 'Cody Bellinger', '641355', '1B, OF', 15.0,
       'Smoak Me A Kingery', 'Whit Merrifield', '593160', '2B, OF', 3.0,
       'Smoak Me A Kingery', 'Mike Moustakas', '519058', '3B', 6.0,
       'Smoak Me A Kingery', 'Mitch Haniger', '571745', 'OF', 3.0,
       'Smoak Me A Kingery', 'Kris Bryant', '592178', '3B, OF', 8.0,
       'Smoak Me A Kingery', 'Michael Conforto', '624424', 'OF', -3.0,
       'Smoak Me A Kingery', 'Scott Kingery', '664068', '2B, 3B, OF',
       -3.0, 'Smoak Me A Kingery', 'Justin Smoak', '475253', '1B', -1.0,
       'Smoak Me A Kingery', 'Corey Dickerson', '572816', 'OF', 16.0,
       'Smoak Me A Kingery', 'Jose Quintana', '500779', 'SP', -2.0,
       'Smoak Me A Kingery', 'Zack Wheeler', '554430', 'SP', 2.0,
       'Smoak Me A Kingery', 'Jack Flaherty', '656427', 'SP', -2.0,
       'Flyin Castillo Bros.', 'Joey Votto', '458015', '1B', 5.0,
       'Flyin Castillo Bros.', '

In [282]:
total_points = {}
teams = []
for i in points5112018:
    teams.append(i)
pos = ['C','1B','2B','3B','SS','OF','SP','RP']
for i in points5112018:
    for n in range(0,len(pos)):
        if len(points5112018[i][pos[n]]) == 1:
            total_points[i] = {pos[n] : points5112018[i][pos[n]][0][2]}

In [283]:
pos

['C',
 'RP',
 'SS',
 'OF',
 'SP',
 '1B',
 '2B',
 '3B',
 'C',
 'RP',
 'SS',
 'OF',
 'SP',
 '1B',
 '2B',
 '3B',
 'C',
 'RP',
 'SS',
 'OF',
 'SP',
 '1B',
 '2B',
 '3B']

In [234]:
len(points5112018['Smoak Me A Kingery']['C']) == 1

True

In [270]:
total_points

{'Flyin Castillo Bros.': {'SS': 10.0}, 'Smoak Me A Kingery': {'SS': 0.0}}

In [29]:
def points2(dataframe):
    points_dict = {}
    teams = []
    for i in set(dataframe.Team):
        teams.append(i)
    for n in range(0,len(teams)):
        points_dict[teams[n]] = {'players': []} 
    for index, row in dataframe.iterrows():
        points_dict[row['Team']]['players'].append([row['name'], row['yahoo_pos'], row['points']])
    def takethird(list):
        return list[2]
    for i in points_dict:
        points_dict[i]['players'].sort(key = takethird, reverse = True)
    return points_dict

In [30]:
pointstry = points2(stats5112018)
pointstry

{'Flyin Castillo Bros.': {'players': [['Luke Weaver', 'SP', 20.0],
   ['Ozzie Albies', '2B', 17.0],
   ['Trea Turner', 'SS', 10.0],
   ['Joey Votto', '1B', 7.0],
   ['Teoscar Hernandez', 'OF', 1.0],
   ['Kyle Seager', '3B', 0.0],
   ['Ronald Acuna', 'OF', 0.0],
   ['Carlos Carrasco', 'SP', 0.0],
   ['Bud Norris', 'SP, RP', 0.0],
   ['Brad Boxberger', 'RP', 0.0],
   ['Chad Green', 'SP, RP', 0.0],
   ['Walker Buehler', 'SP', 0.0],
   ['J.A. Happ', 'SP', 0.0],
   ['Luis Castillo', 'SP', 0.0],
   ['Zack Godley', 'SP', 0.0],
   ['Garrett Richards', 'SP', 0.0],
   ['Luiz Gohara', 'SP', 0.0],
   ['Madison Bumgarner', 'SP', 0.0],
   ['Alex Reyes', 'SP, RP', 0.0],
   ['Rhys Hoskins', '1B, OF', -2.0],
   ['Kenta Maeda', 'SP', -2.0],
   ['Freddie Freeman', '1B, 3B', -3.0],
   ['Jose Martinez', '1B, OF', -3.0],
   ['Welington Castillo', 'C', -4.0]]},
 'Red Bat and Blue Bat': {'players': [['Manny Machado', '3B, SS', 32.0],
   ['Salvador Perez', 'C', 13.0],
   ['Jameson Taillon', 'SP', 12.0],
   ['D

In [136]:
pointstry['Smoak Me A Kingery']['players'][0]

['Kris Bryant', '3B, OF', 21.0]

In [167]:
total_points = {}
teams = []
for i in pointstry:
    teams.append(i)
    total_points[i] = {'points': {'C': [],
                                  '1B': [],
                                  '2B': [],
                                  '3B': [],
                                  'SS': [],
                                  'OF': [],
                                  'SP': [],
                                  'RP':[]
                                 },
                      }
pos = ['C','1B','2B','3B','SS','OF','SP','RP']
used_players = []
for i in pointstry:
    for m in range(0,len(pointstry[i]['players'])):
        for n in range(0,len(pos)):
            if pos[n] in pointstry[i]['players'][m][1] and len(points5112018[i][pos[n]]) == 1:
                total_points[i]['points'][pos[n]].append([pointstry[i]['players'][m][0], pointstry[i]['players'][m][2]])
                used_players.append(pointstry[i]['players'][m])
            elif pos[n] in pointstry[i]['players'][m][1] and pointstry[i]['players'][m] not in used_players and len(pointstry[i]['players'][m][1]) <= 2:
                total_points[i]['points'][pos[n]].append([pointstry[i]['players'][m][0], pointstry[i]['players'][m][2]])
                used_players.append(pointstry[i]['players'][m])
            #elif pos[n] in pointstry[i]['players'][m][1] and pointstry[i]['players'][m] not in used_players and poointstry[i]['players'][m][] 
                    
            else:
                print 'no'


no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
n

In [168]:
total_points

{'Flyin Castillo Bros.': {'points': {'1B': [['Joey Votto', 7.0]],
   '2B': [['Ozzie Albies', 17.0]],
   '3B': [['Kyle Seager', 0.0]],
   'C': [['Welington Castillo', -4.0]],
   'OF': [['Teoscar Hernandez', 1.0], ['Ronald Acuna', 0.0]],
   'RP': [['Brad Boxberger', 0.0]],
   'SP': [['Luke Weaver', 20.0],
    ['Carlos Carrasco', 0.0],
    ['Walker Buehler', 0.0],
    ['J.A. Happ', 0.0],
    ['Luis Castillo', 0.0],
    ['Zack Godley', 0.0],
    ['Garrett Richards', 0.0],
    ['Luiz Gohara', 0.0],
    ['Madison Bumgarner', 0.0],
    ['Kenta Maeda', -2.0]],
   'SS': [['Trea Turner', 10.0]]}},
 'Red Bat and Blue Bat': {'points': {'1B': [['Paul Goldschmidt', 0.0],
    ['Carlos Santana', -4.0],
    ['Hanley Ramirez', -4.0]],
   '2B': [['Derek Dietrich', 10.0], ['DJ LeMahieu', 3.0]],
   '3B': [['Justin Turner', 0.0]],
   'C': [['Salvador Perez', 13.0]],
   'OF': [['Kevin Pillar', 8.0], ['Bryce Harper', 0.0]],
   'RP': [['Keone Kela', 9.0],
    ['Brandon Kintzler', 4.0],
    ['Addison Reed', 0.0

In [146]:
pointstry

{'Flyin Castillo Bros.': {'players': [['Luke Weaver', 'SP', 20.0],
   ['Ozzie Albies', '2B', 17.0],
   ['Trea Turner', 'SS', 10.0],
   ['Joey Votto', '1B', 7.0],
   ['Teoscar Hernandez', 'OF', 1.0],
   ['Kyle Seager', '3B', 0.0],
   ['Ronald Acuna', 'OF', 0.0],
   ['Carlos Carrasco', 'SP', 0.0],
   ['Bud Norris', 'SP, RP', 0.0],
   ['Brad Boxberger', 'RP', 0.0],
   ['Chad Green', 'SP, RP', 0.0],
   ['Walker Buehler', 'SP', 0.0],
   ['J.A. Happ', 'SP', 0.0],
   ['Luis Castillo', 'SP', 0.0],
   ['Zack Godley', 'SP', 0.0],
   ['Garrett Richards', 'SP', 0.0],
   ['Luiz Gohara', 'SP', 0.0],
   ['Madison Bumgarner', 'SP', 0.0],
   ['Alex Reyes', 'SP, RP', 0.0],
   ['Rhys Hoskins', '1B, OF', -2.0],
   ['Kenta Maeda', 'SP', -2.0],
   ['Freddie Freeman', '1B, 3B', -3.0],
   ['Jose Martinez', '1B, OF', -3.0],
   ['Welington Castillo', 'C', -4.0]]},
 'Red Bat and Blue Bat': {'players': [['Manny Machado', '3B, SS', 32.0],
   ['Salvador Perez', 'C', 13.0],
   ['Jameson Taillon', 'SP', 12.0],
   ['D

In [157]:
avail_players = []
for i in pointstry:
    for j in pointstry[i]['players']:
        avail_players.append([j][0][0])
avail_players

['Kris Bryant',
 'Michael Conforto',
 'Mike Moustakas',
 'Whit Merrifield',
 'Scott Kingery',
 'Cody Bellinger',
 'Corey Seager',
 'Mitch Haniger',
 'Corey Dickerson',
 'Gerrit Cole',
 'Edwin Diaz',
 'Ken Giles',
 'Zach Davies',
 'Sean Newcomb',
 'James Paxton',
 'Jake Junis',
 'Jose Quintana',
 'Zack Wheeler',
 'Jack Flaherty',
 'Gary Sanchez',
 'Lance Lynn',
 'Justin Smoak',
 'Manny Machado',
 'Salvador Perez',
 'Jameson Taillon',
 'Derek Dietrich',
 'Keone Kela',
 'Kevin Pillar',
 'Buster Posey',
 'Brandon Kintzler',
 'DJ LeMahieu',
 'Marwin Gonzalez',
 'Paul Goldschmidt',
 'Bryce Harper',
 'Justin Turner',
 'Alex Wood',
 'Miles Mikolas',
 'Addison Reed',
 'Archie Bradley',
 'Mike Clevinger',
 'Marcus Stroman',
 'Jeff Samardzija',
 'Chris Archer',
 'Jimmy Nelson',
 'Carlos Santana',
 'Hanley Ramirez',
 'Luke Weaver',
 'Ozzie Albies',
 'Trea Turner',
 'Joey Votto',
 'Teoscar Hernandez',
 'Kyle Seager',
 'Ronald Acuna',
 'Carlos Carrasco',
 'Bud Norris',
 'Brad Boxberger',
 'Chad Gree

70

In [7]:
#save for later for furture player ID look ups

#playerid_list = pd.read_csv('playerid_list.csv')
#for index, row in team_list.iterrows():
#    if playerid_list['mlb_name'].str.contains(row['name']).any():
#        team_list.loc[index, 'id'] = str(playerid_list[playerid_list['mlb_name'] == row['name']]['mlb_id']).split(' ')[4].split('\nName:')[0]
#        team_list.loc[index, 'yahoo_pos'] = str(playerid_list[playerid_list['mlb_name'] == row['name']]['yahoo_pos']).split(' ')[4].split('\nName:')[0]
